In [1]:
%load_ext autoreload
import datetime
# import ete3
import json
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
# import re
import xml.etree.ElementTree as ET
# import xml.dom.minidom
# import xmlschema

In [2]:
%autoreload
from maxes.xes_loader2 import XesLoader, XesLog
# from maxes.xes_file import XesFile
# from maxes.analyze_xes import AnalyzeXes
# import maxes.analyze_sequence
# import maxes.serialization.serialize
# import maxes.graphs
import maxes.utils

In [3]:
maxes.utils.setup_notebook_logging() # RUN ONLY ONCE

In [ ]:
# file = "/vt/md/maxes/maxes/data/Activities of daily living of several individuals_1_all/data/activitylog_uci_detailed_labour.xes/activitylog_uci_detailed_labour.xes"
file = "/vt/md/maxes/maxes/data/Activities of daily living of several individuals_1_all/data/edited_hh104_labour.xes/edited_hh104_labour.xes"
# file = "/vt/md/maxes/maxes/data/Activities of daily living of several individuals_1_all/data/edited_hh104_weekends.xes/edited_hh104_weekends.xes"
# file = "/vt/md/maxes/maxes/data/Activities of daily living of several individuals_1_all/data/edited_hh102_labour.xes/edited_hh102_labour.xes"


loader = XesLoader()
log = loader.load(file)
log

In [5]:
log.df

,concept:name,lifecycle:transition,time:timestamp,work
0,Start,start,2011-06-14 22:33:36+00:00,Start
1,Start,complete,2011-06-14 22:33:36+00:00,Start
2,personalhygiene,start,2011-06-14 22:33:36+00:00,personalhygiene
3,personalhygiene,complete,2011-06-14 22:38:44+00:00,personalhygiene
4,sleep,start,2011-06-14 22:45:24+00:00,sleep
...,...,...,...,...
83,sleep,complete,2011-08-12 21:48:14+00:00,sleep
84,toilet,start,2011-08-12 21:48:15+00:00,toilet
85,toilet,complete,2011-08-12 22:11:26+00:00,toilet
86,End,start,2011-08-12 22:11:26+00:00,End


In [12]:
log.df["time:timestamp"].dt.hour.describe()

count    4200.000000
mean       12.170238
std         6.321679
min         0.000000
25%         7.000000
50%        12.000000
75%        17.000000
max        23.000000
Name: time:timestamp, dtype: float64

In [11]:
# Spans
spans = []
for trace in log.traces:
    time =  trace.df['time:timestamp']
    low = min(time)
    high = max(time)
    span = high - low
    spans.append(span)
pd.Series(spans).describe()

count                           43
mean     0 days 23:27:15.139534883
std      0 days 01:45:29.999562105
min                0 days 14:46:30
25%         0 days 22:51:31.500000
50%                0 days 23:32:58
75%                1 days 00:05:54
max                1 days 04:25:40
dtype: object

In [13]:
# Firsts
firsts = []
for trace in log.traces:
    first =  trace.df['time:timestamp'][0].hour
    firsts.append(first)
pd.Series(firsts).describe()

count    43.000000
mean     21.488372
std       4.119478
min       0.000000
25%      22.000000
50%      22.000000
75%      23.000000
max      23.000000
dtype: float64

In [20]:
df = log.df[['time:timestamp', 'concept:name']].copy()
time = df['time:timestamp']
df['seconds-since-beginning-of-day'] = (time.dt.hour * 60 + time.dt.minute) * 60 + time.dt.second
df

,time:timestamp,concept:name,seconds-since-beginning-of-day
0,2011-06-14 22:33:36+00:00,Start,81216
1,2011-06-14 22:33:36+00:00,Start,81216
2,2011-06-14 22:33:36+00:00,personalhygiene,81216
3,2011-06-14 22:38:44+00:00,personalhygiene,81524
4,2011-06-14 22:45:24+00:00,sleep,81924
...,...,...,...
83,2011-08-12 21:48:14+00:00,sleep,78494
84,2011-08-12 21:48:15+00:00,toilet,78495
85,2011-08-12 22:11:26+00:00,toilet,79886
86,2011-08-12 22:11:26+00:00,End,79886


In [ ]:
from sklearn.model_selection import train_test_split

X, y = df[['seconds-since-beginning-of-day']], df['concept:name']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression
model = LogisticRegression().fit(X_train, y_train)
pred = model.predict(X_test)

from sklearn.metrics import ConfusionMatrixDisplay
fig, ax = plt.subplots(figsize=(10, 5))
ConfusionMatrixDisplay.from_predictions(y_test, pred, ax=ax)

ax.tick_params(axis='x', labelrotation=90)

In [31]:
df['concept:name'].value_counts()

concept:name
toilet             908
sleep              616
work               460
watchtv            456
eatingdrinking     246
personalhygiene    226
outdoors           216
mealpreperation    210
medication         164
cleaning           144
dress               98
snack               94
relax               88
Start               86
End                 86
groom               46
phone               28
bathe               20
entertainguests      8
Name: count, dtype: int64

In [32]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score
accuracy_score(y_test, pred)
# balanced_accuracy_score(y_test, pred)

0.19714285714285715